# Zhang's Algorithm For Camera Calibration

### Import Statements

In [1]:
import os
from camera_callibration_helper import *
import cv2
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


### Load the Images
* raw_img_list (list): list of 40 BGR input images
* grey_img_list (list): list of 40 grey scale input images
* img_labels (list): list of 40 image filenames (mainly for debugging)

In [2]:
# given_data_path = 'C:\\Users\jo_wang\Desktop\ECE661\HW08\Dataset1'
given_data_path = "/Users/wang3450/Desktop/ECE661/HW08/Dataset1"
raw_img_list, grey_img_list, img_labels = loadImages(given_data_path)
assert(len(grey_img_list) == 40)
assert(len(raw_img_list) == 40)
assert(len(img_labels) == 40)

x = img_labels.index('Pic_1.jpg')
y = img_labels.index('Pic_5.jpg')
z = img_labels.index('Pic_10.jpg')
w = img_labels.index('Pic_34.jpg')

print(x,y,z,w)

0 35 1 27


### Apply Canny Edge Detector On Grey Scale Images
* edge_img_list (list): list of edge maps from Canny

In [3]:
edge_img_list = performCanny(grey_img_list)
assert(len(edge_img_list) == 40)

### Apply Hough Transform To all the Images
* hough_lines_list (list): list of 40 images after applying hough transform

In [4]:
hough_lines_list = performHoughTransform(edge_img_list)
assert(len(hough_lines_list) == len(edge_img_list))

### Get the corner points from selected images
* all_corners (list): at each index, list of 80 corner points
* the_chosen_one (list): index of images to use

In [ ]:
the_chosen_one = [0, 35, 1, 27]


all_corners = list()
for i in the_chosen_one:
    h_lines, v_lines = get_Horizontal_Vert_Lines(hough_lines_list[i])

    v_lines = np.array(v_lines).reshape(-1,2)
    h_lines = np.array(h_lines).reshape(-1,2)

    img = deepcopy(raw_img_list[i])
    corner_points = getCorners(v_lines, h_lines)
    if len(corner_points) == 80:
        all_corners.append(corner_points)

    for j, point in enumerate(corner_points):
        try:
            img = cv2.circle(img, point, 3, (0, 0, 255), -1)
            cv2.putText(img, str(j), (point[0]+5, point[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
        except OverflowError:
            pass

    cv2.imwrite(f'points_{i}.jpg', img)

### Get world point coordinates
* world_points (list): list of 80 world point coordinates in sorted order
* Assumption made: squares are 20 pixels apart

In [6]:
world_points = list()
for i in range(0, 160, 20):
    for j in range(0, 200, 20):
        world_points.append([i,j])

### Estimate Homographies between world points and all corners
* all_homographies (list): list of 3x3 homographies relating world points to each image
* DON'T DELETE THIS ONE CUZ IT WORKS FOR NOW!!!!!!

In [7]:
all_homographies = list()
for corners in all_corners:
    h = get_H(world_points, corners)
    all_homographies.append(h)

### Compute W
* W is a 3x3 matrix
* Derived from the solution of Vb = 0
* Use svd to solve Vb=0

In [8]:
Big_V = np.zeros((1,6))
for h in all_homographies:
    r1 = get_V(i=1, j=2, h=h).T
    r2 = get_V(i=1,j=1,h=h).T - get_V(i=2,j=2,h=h).T
    Big_V = np.vstack((Big_V, r1))
    Big_V = np.vstack((Big_V, r2))

Big_V = Big_V[1:, :]

u, s, vh = np.linalg.svd(Big_V)
b = vh[-1]

w = np.zeros((3,3))
w[0][0] = b[0]
w[0][1] = b[1]
w[0][2] = b[3]
w[1][0] = b[1]
w[1][1] = b[2]
w[1][2] = b[4]
w[2][0] = b[3]
w[2][1] = b[4]
w[2][2] = b[5]

### Compute Intrinsic Camera Parameters Matrix k
* k is 3x3 matrix
* k is based on y0, a_x, a_y, skew, x0, lambda


In [9]:
y0 = ((w[0][1] * w[0][2]) - (w[0][0] * w[1][2])) / (w[0][0] * w[1][1] - w[0][1] ** 2)
scale_lambda = w[2][2] - (w[0][2] ** 2 + y0 * (w[0][1] * w[0][2] - w[0][0] * w[1][2])) / w[0][0]
a_x = np.sqrt(np.abs((scale_lambda / w[0][0])))
a_y = np.sqrt(np.abs((scale_lambda * w[0][0]) / (w[0][0] * w[1][1] - w[0][1] **2)))
skew = (-1 * w[0][1] * (a_x ** 2) * a_y) / scale_lambda
x0 = (skew * y0) / a_y - (w[0][2] * (a_x ** 2)) / scale_lambda

k = np.zeros((3,3))
k[0][0] = a_x
k[0][1] = skew
k[0][2] = x0
k[1][1] = a_y
k[1][2] = y0
k[2][2] = 1

print(k)

[[7.27611186e+02 2.60919073e-02 3.20441973e+02]
 [0.00000000e+00 7.26434435e+02 2.42237902e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


### Compute Extrinsic Parameters

In [10]:
def get_extrinsic(k, h):
    zeta = 1 / np.linalg.norm(np.linalg.inv(k) @ h[:,0])

    r1 = zeta * np.linalg.inv(k) @ h[:,0]
    r2 = zeta * np.linalg.inv(k) @ h[:,1]
    r3 = zeta * np.cross(r1,r2)
    t = zeta * np.linalg.inv(k) @ h[:,2]

    r1 = np.reshape(r1, (3,1))
    r2 = np.reshape(r2, (3,1))
    r3 = np.reshape(r3, (3,1))
    t = np.reshape(t, (3,1))

    R = np.hstack((r1,r2))
    R = np.hstack((R, r3))
    R = np.reshape(R, (3,3))

    u, _, vh = np.linalg.svd(R)

    R = u @ vh

    return R, t

all_rotations = list()
all_translations = list()

for homographies in all_homographies:
    R, t = get_extrinsic(k, homographies)
    all_rotations.append(R)
    all_translations.append(t)

assert(len(all_rotations) == len(all_translations))
assert(len(all_rotations) == len(the_chosen_one))

print("Pic 1")
print(f'Rotation Matrix: \n{all_rotations[0]}')
print(f'Translation Matrix: \n {all_translations[0]}')
print("\n")
print("Pic 5")
print(f'Rotation Matrix: \n{all_rotations[1]}')
print(f'Translation Matrix: \n {all_translations[1]}')
print("\n")
print("Pic 10")
print(f'Rotation Matrix: \n{all_rotations[2]}')
print(f'Translation Matrix: \n {all_translations[2]}')
print("\n")
print("Pic 34")
print(f'Rotation Matrix: \n{all_rotations[3]}')
print(f'Translation Matrix: \n {all_translations[3]}')
print("\n")

Pic 1
Rotation Matrix: 
[[ 0.78755375 -0.18504673  0.58780677]
 [ 0.19763533  0.97930997  0.04350012]
 [-0.58369458  0.0819127   0.80783101]]
Translation Matrix: 
 [[ -37.29988982]
 [-103.08370954]
 [ 441.81089493]]


Pic 5
Rotation Matrix: 
[[ 0.98777016 -0.15501223  0.01677272]
 [ 0.15578832  0.98559751 -0.06578405]
 [-0.00633382  0.06759252  0.99769291]]
Translation Matrix: 
 [[-43.01419107]
 [-98.01655764]
 [420.27097024]]


Pic 10
Rotation Matrix: 
[[ 0.74717874  0.1886999  -0.63727253]
 [ 0.13193634  0.89765281  0.42049046]
 [ 0.65139599 -0.39826094  0.64581073]]
Translation Matrix: 
 [[-58.55024996]
 [-95.95843495]
 [426.46924173]]


Pic 34
Rotation Matrix: 
[[ 0.93722295 -0.12121838 -0.32698508]
 [ 0.08510211  0.98879608 -0.12263744]
 [ 0.33818748  0.08711151  0.93703832]]
Translation Matrix: 
 [[-59.12062571]
 [-99.19596354]
 [314.35583947]]




### Reproject the World Coordinates

In [11]:
#the_chosen_one = [0, 35, 1, 27]
corner0 = [list(i) for i in all_corners[0]]
corner1 = [list(i) for i in all_corners[1]]
corner2 = [list(i) for i in all_corners[2]]
corner3 = [list(i) for i in all_corners[3]]

rep_img0, rep_img0_mean_e, rep_img0_var_e = ReprojectPoints(raw_img_list[0],world_points,corner0,k,all_rotations[0],all_translations[0])

rep_img1, rep_img1_mean_e, rep_img1_var_e = ReprojectPoints(raw_img_list[35],world_points,corner1,k,all_rotations[1],all_translations[1])

rep_img2, rep_img2_mean_e, rep_img2_var_e = ReprojectPoints(raw_img_list[1],world_points,corner2,k,all_rotations[2],all_translations[2])

rep_img3, rep_img3_mean_e, rep_img3_var_e = ReprojectPoints(raw_img_list[27],world_points,corner3,k,all_rotations[3],all_translations[3])

cv2.imwrite('rep_pic1.jpg', rep_img0)
cv2.imwrite('rep_pic5.jpg', rep_img1)
cv2.imwrite('rep_pic10.jpg', rep_img2)
cv2.imwrite('rep_pic34.jpg', rep_img3)

print('Pic #     Mean Error             Error Variance')
print(f'Pic_1    {rep_img0_mean_e}      {rep_img0_var_e}')
print(f'Pic_5    {rep_img1_mean_e}      {rep_img1_var_e}')
print(f'Pic_10   {rep_img2_mean_e}      {rep_img2_var_e}')
print(f'Pic_34   {rep_img3_mean_e}      {rep_img3_var_e}')

Pic #     Mean Error             Error Variance
Pic_1    1.2124966087180533      0.4419708546416626
Pic_5    0.6834195902286995      0.11333566797308778
Pic_10   0.8883294030357984      0.2100304351027858
Pic_34   1.0073327588105954      0.2513307337281006
